In [13]:
%run '03_Parabolic_Subgroup.ipynb'

In [14]:
class Homogeneous_Variety ( Algebraic_Structure ) :

    BASIS_LABELLING = { 'sr' : [ 'sr' , 'simple roots'        , 'alphas'                                                                     , 'A' ] ,
                        'mx' : [ 'mx' , 'mixed'               , 'simple roots for included nodes and fundamental weights for excluded nodes' , 'B' ] ,
                        'fw' : [ 'fw' , 'fundamental weights' , 'omegas'                                                                     , 'C' ]
                      }


In [15]:
class Irreducible_Homogeneous_Variety ( Irreducible_Algebraic_Structure , Homogeneous_Variety ) :

    def __init__( self , Parabolic_Subgroup:Parabolic_Subgroup_In_Irreducible_Cartan_Group ) -> None :
        """
        Initialize a homogeneous variety X=G/P.

        INPUT:
        - ``Parabolic_Subgroup`` -- Parabolic_Subgroup_In_Irreducible_Cartan_Group ;

        OUTPUT: None.
        """
        assert isinstance( Parabolic_Subgroup , Parabolic_Subgroup_In_Irreducible_Cartan_Group ) , \
               TypeError('The input must be a parabolic subgroup in an irreducible Cartan group.')
        self._Parabolic_Subgroup = Parabolic_Subgroup


    def __repr__ ( self ) -> Parabolic_Subgroup_In_Irreducible_Cartan_Group :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Parent_Group()


    def __str__ ( self , Output_Style='Long' ) -> str :
        """Returns a one-line string as short description."""
        if   Output_Style == 'Short' : return self.Parent_Group().Cartan_String() + '/P(' + str( self.Included_Nodes() ) + ')'
        elif Output_Style == 'Long'  : return 'Smooth projective variety ' + self.__str__( Output_Style='Short' ) + '.'
        else                         : raise ValueError('The input for ``Output_Style`` is inappropriate.')


    def Available_Negative_Roots ( self ) -> "Root" :
        """Returns all available negative roots."""
        for Root in self.Parent_Group().Cartan_Type().root_system().root_lattice().negative_roots() :
            if not Root in self.Parabolic_Subgroup().Negative_Roots() :
                yield Root


    def Available_Positive_Roots ( self ) -> "Root" :
        """Returns all available positive roots."""
        for Root in self.Parent_Group().Cartan_Type().root_system().root_lattice().positive_roots() :
            if not Root in self.Parabolic_Subgroup().Negative_Roots() :
                yield Root


    def Available_Roots ( self ) -> "Root" :
        """Returns all available roots."""
        for Root in self.Parent_Group().Cartan_Type().root_system().root_lattice().roots() :
            if not Root in self.Parabolic_Subgroup().Negative_Roots() :
                yield Root


    def Basis ( self , Label:str='mx' ) -> dict :
        """
        Returns the basis in terms of simple roots and fundamental weights.

        INPUT:
        - ``self`` -- Parabolic_Subgroup; the parabolic subgroup P of a Cartan group G.
        - ``Label`` -- str (default: 'mx');

        OUTPUT:
        - ``Output`` -- Dictionary.

        ALGORITHM:
        - alpha_i : the simple root associated to the i-th node.
        - omega_i : the fundamental weight associated to the i-th node.

        - sr = { alpha_i }
        - mx_{ i : i excluded } = { alpha_i : i included } cup { omega_i : i excluded }.
          Note: mx_{} = sr and mx_{ all i's } = fw
        - fw = { omega_i }
        """
        if   Label in self.BASIS_LABELLING['sr'] :
            return dict( self.Parent_Group().Cartan_Type().root_system().weight_space().simple_roots() )

        elif Label in self.BASIS_LABELLING['mx'] :
            MixedBasis = {}
            for Node in self.Parent_Group().Cartan_Type().index_set() :
                if Node in self.Parabolic_Subgroup().Included_Nodes() : Element = self.Parent_Group().Cartan_Type().root_system().weight_space().simple_root(Node)
                elif Node in self.Parabolic_Subgroup().Excluded_Nodes() : Element = self.Parent_Group().Cartan_Type().root_system().weight_space().fundamental_weight(Node)
                else : raise ValueError('Plausibility? The set of nodes subdivides into the disjoint union of marked ones and excluded ones.')
                MixedBasis.update( { Node : Element } )
            return MixedBasis

        elif Label in self.BASIS_LABELLING['fw'] :
            return dict( self.Parent_Group().Cartan_Type().root_system().weight_space().fundamental_weights() )

        else : raise ValueError('The input of ``Label`` is inappropriate.')


    # Synonym for the method ``Tautological_Subbundle``
    def calO ( self , *Twists:tuple[int] ) -> "Irreducible_Equivariant_Vector_Bundle" :
        """Returns the structure sheaf or a twist of it."""
        return self.Structure_Sheaf( *Twists )


    # Snynoms for different methods depending on the input
    def calU ( self , *Input ) -> "Irreducible_Equivaraint_Vector_Bundle" :
        """Returns the tautological subbundle (if there no input) or the irreequivariant vector bundle associated to a highest weight (if there is some input otherwise)."""
        if len(Input) == 0 : # Synonym for the method ``Tautological_Subbundle``
            return self.Tautological_Subbundle()

        else : # Synonym for the method ``Equivariant_Vector_Bundles``
            return self.Equivariant_Vector_Bundle( *Input )


    # Synonym for the method ``Trivial_Vector_Bundle``
    def calV ( self ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the trivial vector bundle ."""
        return self.Trivial_Vector_Bundle()


    def Cartan_Degree ( self ) -> int :
        """Returns the Cartan degree of the parent Group G."""
        return self.Parent_Group().Cartan_Degree()


    def Cartan_Family ( self ) -> str :
        """Returns the Cartan family of the parent Group G."""
        return self.Parent_Group().Cartan_Family()


    def Cartan_Matrix ( self , From:str='fw' , To:str='mx' ) -> Matrix :
        """
        Returns the base change between the bases sr, mx or fw.

        INPUT:
        - ``self`` -- Parabolic_Subgroup; the parabolic subgroup P of a Cartan group G.

        OUTPUT:
        - ``Output`` -- Matrix.

        ALGORITHM:
        - alpha_i : the simple root associated to the i-th node.
        - omega_i : the fundamental weight associated to the i-th node.

        - sr >> fw : Cartan matrix
        - fw >> sr : Inverse of Cartan matrix (sr >> fw)
        - mx >> fw : For each excluded node i, replace the i-th column by a standard vector e_i.
        - fw >> mx : Inverse of mx >> fw
        - sr >> mx : Composition of sr >> fw >> mx
        - mx >> sr : Inverse of sr >> mx
        """
        if   From in self.BASIS_LABELLING['sr'] : From = 'sr'
        elif From in self.BASIS_LABELLING['mx'] : From = 'mx'
        elif From in self.BASIS_LABELLING['fw'] : From = 'fw'
        else : raise ValueError('The input for ``From`` is inappropriate.')
        if   To   in self.BASIS_LABELLING['sr'] : To = 'sr'
        elif To   in self.BASIS_LABELLING['mx'] : To = 'mx'
        elif To   in self.BASIS_LABELLING['fw'] : To = 'fw'
        else : raise ValueError('The input for ``To`` is inappropriate.')

        if   From == To : return matrix.identity( self.Parent_Group().Cartan_Degree() )

        elif From == 'sr' :
            if   To == 'mx' : return self.Cartan_Matrix( From = 'fw' , To = 'mx') * self.Cartan_Matrix( From = 'sr' , To = 'fw')
            elif To == 'fw' : return self.Parent_Group().Cartan_Type().cartan_matrix()

        elif From == 'mx' :
            if   To == 'sr' : return self.Cartan_Matrix( From = 'sr' , To = 'mx' ).inverse()
            elif To == 'fw' :
                Matrix = copy( self.Cartan_Matrix( From = 'sr' , To = 'fw' ) )
                for Node in self.Parabolic_Subgroup().Excluded_Nodes() :
                    Matrix[:,Node-1] = vector([ kronecker_delta( I , Node ) for I in [ 1 .. self.Parent_Group().Cartan_Degree() ] ])
                return Matrix

        elif From == 'fw' :
            if   To == 'sr' : return self.Cartan_Matrix( From = 'sr' , To = 'fw' ).inverse()
            elif To == 'mx' : return self.Cartan_Matrix( From = 'mx' , To = 'fw' ).inverse()


    def Cartan_String ( self ) -> str :
        """Returns the Cartan string of the parent Group G."""
        return self.Parent_Group().Cartan_String()


    def Cartan_Type ( self ) -> CartanType :
        """Returns the attribute ``Cartan_Type`` of the parent Group G."""
        return self.Parent_Group().Cartan_Type()


    def Compare_Roots ( self , Root1 , Root2 ) -> str or None :
        """Returns None if the two roots are not comparable and a result '<', '==', or '>' otherwise."""

        #CAN BE DELETED!
        #Available_Roots = [ Root for Root in self.Available_Roots() ]
        #assert Root1 in Available_Roots , \
        #       ValueError('The input for ``Root1`` is not an available root.')
        #assert Root2 in Available_Roots , \
        #       ValueError('The input for ``Root2`` is not an available root.')
        #
        #Difference = Root1 - Root2
        #
        ## Test if the difference has only a support over the included nodes
        #if not set( Difference.support() ).issubset( set(self.Parabolic_Subgroup().Included_Nodes()) ) : return None
        #
        ## Test if there is a common signum
        #Signa = set([ sgn(Coefficient) for Coefficient in Difference.coefficients() ])
        #if   len(Signa) == 0 : return '=='
        #elif len(Signa) == 1 :
        #    if    1 in Signa : return '>'
        #    elif -1 in Signa : return '<'
        #elif len(Signa) == 2 : return None

        RS = Y.Parent_Group().Cartan_Type().root_system().root_space()
        assert Root1 in RS , \
               ValueError('The input for ``Root1`` is not a root.')
        assert Root2 in RS , \
               ValueError('The input for ``Root2`` is not a root.')

        if Root1 == Root2 :
            return '=='

        else :
            Difference_In_srBasis = Root1-Root2
            if        Difference_In_srBasis.is_positive_root() : return '>'
            elif (-1*Difference_In_srBasis).is_positive_root() : return '<'
            else                                               : return None


    def Compare_Weights ( self , Weight1 , Weight2 ) -> str or None :
        """Returns None if the two weights are not comparable and a result '<', '==', or '>' otherwise."""

        WS = self.Parent_Group().Cartan_Type().root_system().weight_space()
        assert Weight1 in WS , \
               ValueError('The input for ``Weight1`` is not a weight.')
        assert Weight2 in WS , \
               ValueError('The input for ``Weight2`` is not a weight.')

        if Weight1 == Weight2 :
            return '=='

        else :
            RL = self.Parent_Group().Cartan_Type().root_system().root_space()
            scr = RL.simple_coroots()
            sr = RL.simple_roots()
            Coefficients_For_fwBasis = [ (Weight1-Weight2).scalar(coroot) for coroot in scr ]
            Coefficients_For_srBasis = vector( QQ , Coefficients_For_fwBasis ) * self.Cartan_Matrix( From='fw' , To='sr' )
            Difference_In_srBasis = sum([ QQ(Coefficient)*sr[Node] for Node , Coefficient in enumerate( list( Coefficients_For_srBasis ) , start = 1 ) ])

            if        Difference_In_srBasis.is_positive_root() : return '>'
            elif (-1*Difference_In_srBasis).is_positive_root() : return '<'
            else                                               : return None


    def Complex ( self , Objects ) -> "Complex_Over_Irreducible_Homogeneous_Variety" :
        """Returns a complex of equivariant vector bundles over ``self``."""
        return Complex_Of_Coherent_Sheaves( Base_Space=self , Objects=Objects )


    # ..ToDo: For PP^n, it is only correct up to sign. 
    def Complex_Defining_Exterior_Power_Of_Tautological_Quotient_Bundle ( self , p :int =1 ) -> "Complex_Over_Irreducible_Homogeneous_Variety" :
        """Returns the complex defining the p-th exterior power of the tautological quotient bundle on ``self``."""
        assert isinstance( p , Integer ) , \
               'The input for `p` need to be an integer.'

        G = self.Parent_Group()
        fw = self.Basis('fw')
        Terms = dict({})
        for i in [ 0 .. p+1 ] :
            if i == 0 : Terms.update({ 0    : 'Cokernel' })
            else      : Terms.update({ -1*i : self.Tautological_Subbundle().Symmetric_Power( i-1 ).Multiply_By( G.rmV( fw[1] ).exterior_power( p-i+1 ) ) })
        return self.Complex( Terms )


    def Dimension ( self ) -> int :
        """Returns the dimension dim X = dim G - dim P."""
        return self.Parent_Group().Dimension() - self.Parabolic_Subgroup().Dimension()


    def Equivariant_Vector_Bundle ( self , *Inputs:tuple ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the direct sum of equivariant vector bundles induced by the given highest weights."""
        Constituent_Parts = list(Inputs)
        return Equivariant_Vector_Bundle.Constructor( Base_Space=self , Constituent_Parts=Constituent_Parts )


    def Excluded_Nodes ( self ) -> set :
        """Returns the attribute ``Excluded_Nodes`` of the parabolic subgroup P."""
        return self.Parabolic_Subgroup().Excluded_Nodes()


    def Expected_Support_Partition_For_Minimal_Full_Lefschetz_Collection ( self ) -> tuple[ int ] :
        """Returns the expected support partition for a minimal Lefschetz collection on ``self``."""
        assert self.Parabolic_Subgroup().Is_Maximal() , \
               ValueError('The method is only implemented for those homogeneous varieties where the parabolic subgroup is maximal.')

        Collection__Expected_Length = self.K0().rank()
        Node , Fano_Index = next(self.Fano_Index())

        Rectangular_Part__Expected_Height = Fano_Index
        Rectangular_Part__Expected_Width = floor( Collection__Expected_Length / Rectangular_Part__Expected_Height )
        Residual_Part__Expected_Height = Collection__Expected_Length - Rectangular_Part__Expected_Height * Rectangular_Part__Expected_Width
        if Residual_Part__Expected_Height == 0 : Residual_Part__Expected_Width = 0
        else                                   : Residual_Part__Expected_Width = 1

        return tuple(                                    Residual_Part__Expected_Height * [ Rectangular_Part__Expected_Width+1 ] + \
                     (Rectangular_Part__Expected_Height-Residual_Part__Expected_Height) * [ Rectangular_Part__Expected_Width   ]   \
                    )


    def Fano_Index ( self ) -> Iterator[ tuple[ int , int ] ]:
        """
        Returns the Fano index of ``self``.

        INPUT:
        - ``self`` -- Parabolic_Subgroup; the parabolic subgroup P of a Cartan group G.

        OUTPUT:
        - ``Node`` -- Excluded node
        - ``Fano_Index`` -- Index associated to X = G/P(Excluded_Nodes={Node})

        ALGORITHM:
        Thanks to the post by Pieter Belmans concerning the index of partial flag varieties
        from Aug 23rd, 2018 on his blog (cf. to [Blog_PieterBelmans]_). The link is
        https://pbelmans.ncag.info/blog/2018/08/23/index-partial-flag-varieties/
        (Date: Apr 26th, 2021).

        The index, i.e. for X=G/P (P maximal), we have Pic(X) ≅ ZZ⋅O_X(1) and therefore define the index as the integer i
        such that ω∨_X ≅ O_X(1) ⊗ i.

        To compute it, we use lemma 2.19 and remark 2.20 of [KP2016]. Combined they say the following:
        Let β be the simple root corresponding to the chosen maximal parabolic subgroup P, and ξ the associated
        fundamental weight. Let ¯β be the maximal root of the same length as β such that the coefficient of β
        in the expression of ¯β is 1.
        Then the index of G/P equals i_G/P = (ρ,β+¯β)/(ξ,β).

        REFERENCE:
        [Blog_PieterBelmans] https://pbelmans.ncag.info/blog/
        [KP2016] Kuznetsov, Alexander; Polishchuk, Alexander Exceptional collections on isotropic Grassmannians.
                 J. Eur. Math. Soc. (JEMS) 18 (2016), no. 3, 507–574.
        """
        Root_Lattice = self.Parent_Group().Cartan_Type().root_system().root_lattice()
        Ambient_Space = self.Parent_Group().Cartan_Type().root_system().ambient_space()

        for Node in self.Parabolic_Subgroup().Excluded_Nodes() :
            Beta = Root_Lattice.simple_root(Node).to_ambient() # The simple root associated to ``Node``
            Xi = Ambient_Space.fundamental_weight(Node) # The fundamental weight associated to ``Node``
            Rho = Ambient_Space.rho() # Sum of fundamental weights
            Length = Beta.dot_product(Beta)
            Beta_Bar = [Alpha for Alpha in Root_Lattice.roots() if Alpha.coefficient(Node) == 1 and Alpha.to_ambient().dot_product(Alpha.to_ambient()) == Length][-1].to_ambient()
                # The maximal root of the same length as ``Beta`` such that the coefficient of ``Beta`` in the expression of ``Beta_Bar`` is 1.
            Fano_Index = Rho.dot_product(Beta + Beta_Bar) / Xi.dot_product(Beta)
            yield Node , Fano_Index


    def First_Collection ( self ) -> "Lefschetz_Collection" :
        assert self.Parabolic_Subgroup().Is_Maximal() , \
               ValueError('The method is only implemented for those homogeneous varieties where the parabolic subgroup is maximal.')
        n = self.Parent_Group().Cartan_Degree()
        k = self.k()
        fw = self.Basis('fw')
        LC = self.Lefschetz_Collection( Starting_Block=[] , Support_Pattern='Trivial' )
        for Degree in [ 0 .. n-k ] :
            for Coefficients in IntegerListsLex( n=Degree , length=k-1 ) :
                Highest_Weight = sum(   [ self.Null_Weight() ]
                                      + [ Coefficient * fw[Node] for Node , Coefficient in enumerate( Coefficients , start=1 ) ]
                                    )
                LC += self.Irreducible_Equivariant_Vector_Bundle( Highest_Weight ).Maximal_Numerically_Exceptional_Orbit()
        return LC


    def Grothendieck_Group ( self ) -> FreeModule :
        """
        Returns the Grothendieck Group of X=G/P.

        INPUT:
        - ``self`` -- Parabolic_Subgroup; the parabolic subgroup P of a Cartan group G.

        OUTPUT:
        - ``Output`` -- Integer; The rank of the Grothendieck group of X=G/P.

        ALGORITHM:
        Thanks to the post by Pieter Belmans concerning the index of partial flag varieties
        from Aug 22nd, 2018 on his blog (cf. to [Blog_PieterBelmans]_). The link is
        https://pbelmans.ncag.info/blog/2018/08/22/rank-flag-varieties/
        (Date: Apr 26th, 2021).

        It can be computed as the rank of the homology of X=G/P using the Bruhat cell decomposition,
        and is equal to #WG/#WL, where WG (resp. WL) denotes the Weyl group of G (resp. L).
        L is the Levi subgroup inside the maximal parabolic P.

        .. TODO:
        - Check if the rank of the Grothendieck group is always be given by the Euler characteristic.
        - Fix Rank_Of_GrothendieckGroup for Borel subgroups.

        REFERENCE:
        [Blog_PieterBelmans] https://pbelmans.ncag.info/blog/
        [KP2016] Kuznetsov, Alexander; Polishchuk, Alexander Exceptional collections on isotropic Grassmannians.
                 J. Eur. Math. Soc. (JEMS) 18 (2016), no. 3, 507–574.
        """
        Cartan_Type_of_Parent_Group = self.Parent_Group().Cartan_Type()
        Weyl_Group_of_Parent_Group = Cartan_Type_of_Parent_Group.root_system().root_lattice().weyl_group()

        if self.Parabolic_Subgroup().Is_Borel() :
            # Question: Because WeylGroup_of_Levi_Part.cardinality() == 1?
            return Integers()^( Weyl_Group_of_Parent_Group.cardinality() )

        else :
            Cartan_Type_of_Levi_Part = self.Parent_Group().Cartan_Type().dynkin_diagram().subtype( self.Parabolic_Subgroup().Included_Nodes() )
            Weyl_Group_of_Levi_Part = Cartan_Type_of_Levi_Part.root_system().root_lattice().weyl_group()
            return Integers()^( Weyl_Group_of_Parent_Group.cardinality() / Weyl_Group_of_Levi_Part.cardinality() )


    def Included_Nodes ( self ) -> set :
        """Returns the attribute ``Included_Nodes`` of the parabolic subgroup P."""
        return self.Parabolic_Subgroup().Included_Nodes()



    def Irreducible_Equivariant_Vector_Bundle ( self , Highest_Weight:"Weight" ) -> "Irreducible_Equivariant_Vector_Bundle" :
        """Returns the equivariant vector bundle induced by the given highest weight."""
        if Highest_Weight == 0 :
            fw = self.Basis('fw')
            Highest_Weight = 0 * fw[1]
        return Irreducible_Equivariant_Vector_Bundle( Base_Space=self , Highest_Weight=Highest_Weight )


    # ..ToDo : Improve the testing. Get a cleaner algorithm based immediately on the definition.
    def Is_Adjoint ( self ) -> bool :
        """Tests if ``self`` is adjoint (cf. grassmannian.info)"""
        G = self.Parent_Group()
        Cartan_Family = G.Cartan_Family()
        Cartan_Degree = G.Cartan_Degree()
        P = self.Parabolic_Subgroup()
        assert P.Is_Maximal() , \
               ValueError('This method is only implemented for maximal parabolic subgroups.')
        k = self.k()
        if   Cartan_Family == 'B' : return k == 2
        elif Cartan_Family == 'C' : return k == 1
        elif Cartan_Family == 'D' : return k == 2 and 4 <= Cartan_Degree
        elif Cartan_Family == 'E' : return ( Cartan_Degree == 6 and k == 2 ) or ( Cartan_Degree == 7 and k == 1 ) or ( Cartan_Degree == 8 and k == 8 )
        elif Cartan_Family == 'F' : return ( Cartan_Degree == 4 and k == 1 )
        elif Cartan_Family == 'G' : return ( Cartan_Degree == 2 and k == 2 )
        else                      : return False


    # ..ToDo : Improve the testing. Get a cleaner algorithm based immediately on the definition.
    def Is_Coadjoint ( self ) -> bool :
        """Tests if ``self`` is coadjoint (cf. grassmannian.info)"""
        G = self.Parent_Group()
        Cartan_Family = G.Cartan_Family()
        Cartan_Degree = G.Cartan_Degree()
        P = self.Parabolic_Subgroup()
        assert P.Is_Maximal() , \
               ValueError('This method is only implemented for maximal parabolic subgroups.')
        k = self.k()
        if   Cartan_Family == 'B' : return k == 1
        elif Cartan_Family == 'C' : return k == 2
        elif Cartan_Family == 'D' : return k == 2 and 4 <= Cartan_Degree
        elif Cartan_Family == 'E' : return ( Cartan_Degree == 6 and k == 2 ) or ( Cartan_Degree == 7 and k == 1 ) or ( Cartan_Degree == 8 and k == 8 )
        elif Cartan_Family == 'F' : return ( Cartan_Degree == 4 and k == 4 )
        elif Cartan_Family == 'G' : return ( Cartan_Degree == 2 and k == 1 )
        else                      : return False


    # ..ToDo : Improve the testing. Get a cleaner algorithm based immediately on the definition.
    def Is_Cominuscule ( self ) -> bool :
        """Tests if ``self`` is cominuscule (cf. grassmannian.info)"""
        G = self.Parent_Group()
        Cartan_Family = G.Cartan_Family()
        Cartan_Degree = G.Cartan_Degree()
        P = self.Parabolic_Subgroup()
        assert P.Is_Maximal() , \
               ValueError('This function is only implemented for maximal parabolic subgroups.')
        k = self.k()
        if   Cartan_Family == 'A' : return True
        elif Cartan_Family == 'B' : return k == 1
        elif Cartan_Family == 'C' : return k == Cartan_Degree
        elif Cartan_Family == 'D' : return k in [ 1 , CartenDegree-1 , Cartan_Degree ]
        elif Cartan_Family == 'E' : return ( Cartan_Degree == 6 and k in [ 1 , 6 ] ) or ( Cartan_Degree == 7 and k == 7 )
        else                      : return False


    # ..ToDo : Specify the type for the input ``Weight``!
    def Is_Levi_Dominant ( self , Weight ) -> bool :
        """Tests if a given weight on ``self`` is Levi-dominant (i.e. all coefficients over included nodes are non-negative)."""
        assert Weight in self.Parent_Group().Cartan_Type().root_system().weight_space() , \
               ValueError('The weight must be an element of the weight space: '+str(self.Parent_Group().Cartan_Type().root_system().weight_space()))
        for Node in self.Parabolic_Subgroup().Included_Nodes() :
            if Weight.coefficient(Node) < 0 : return False
        return True


    # ..ToDo : Improve the testing. Get a cleaner algorithm based immediately on the definition.
    def Is_Minuscule ( self ) -> bool :
        """Tests if ``self`` is minuscule (cf. grassmannian.info)"""
        G = self.Parent_Group()
        Cartan_Family = G.Cartan_Family()
        Cartan_Degree = G.Cartan_Degree()
        P = self.Parabolic_Subgroup()
        assert P.Is_Maximal() , \
               ValueError('This function is only implemented for maximal parabolic subgroups.')
        k = self.k()
        if   Cartan_Family == 'A' : return True
        elif Cartan_Family == 'B' : return k == Cartan_Degree
        elif Cartan_Family == 'C' : return k == 1
        elif Cartan_Family == 'D' : return k in [ 1 , CartenDegree-1 , Cartan_Degree ]
        elif Cartan_Family == 'E' : return ( Cartan_Degree == 6 and k in [ 1 , 6 ] ) or ( Cartan_Degree == 7 and k == 7 )
        else                     : return False


    def Is_Regular ( self , Weight:"Weight" ) -> bool :
        """Tests if a given weight on ``self`` is G-regular (i.e. does not ly on a wall of a Weyl chamber)."""
        assert Weight in self.Parent_Group().Cartan_Type().root_system().weight_space() , \
               ValueError('The weight must be an element of the weight space: '+str(self.Parent_Group().Cartan_Type().root_system().weight_space()))
        return not self.Is_Singular( Weight )


    def Is_Singular ( self , Weight:"Weight" ) -> bool :
        """Tests if a given weight on ``self`` is G-singular (i.e. lies on a wall of a Weyl chamber)."""
        assert Weight in self.Parent_Group().Cartan_Type().root_system().weight_space() , \
               ValueError('The weight must be an element of the weight space: '+str(self.Parent_Group().Cartan_Type().root_system().weight_space()))
        # Move ``Weight`` to dominant chamber
        Weight_In_Dominant_Chamber = Weight.to_dominant_chamber()
        # Consider list of reflections of ``Weight`` and check whether ``Weight`` is invariant under some reflection
        Reflections = [ Weight_In_Dominant_Chamber.simple_reflection( Index ) for Index in self.Parent_Group().Cartan_Type().index_set() ]
        return Weight_In_Dominant_Chamber in Reflections


    def k ( self ) -> int :
        """Returns the number ``k`` given in the realisation as Grassmannian."""
        P = self.Parabolic_Subgroup()
        Excluded_Nodes = P.Excluded_Nodes(Output_Type=list)
        if P.Is_Maximal() : return Excluded_Nodes[0]
        else              : return Excluded_Nodes


    # Synonym for the method ``Grothendieck_Group``
    def K0 ( self ) -> FreeModule :
        """Returns the Grothendieck Group of X=G/P."""
        return self.Grothendieck_Group()


    def Lefschetz_Collection ( self , Starting_Block , Twist=None , Support_Pattern='Trivial' ) -> "Lefschetz_Collection" :
        """Returns a Lefschetz collection over ``self``."""
        return Lefschetz_Collection( Base_Space=self , Starting_Block=Starting_Block , Twist=Twist , Support_Pattern=Support_Pattern )


    def Mapping ( self , Image:"Irreducible_Homogeneous_Variety" , Dictionary:dict , Output_Type:str='fw' ) -> dict :
        """
        INPUT:
        - ``self`` -- Irreducible homoegenous variety (Source)
        - ``Image`` -- Irreducible homoegenous variety (Image)
        - ``Dictionary`` -- Dictionary (Image); Description of the mapping on the level of simple roots
        - ``Output_Type`` -- str 'fw' (default) or 'sr'; Output on the level of fundamental weights (fw) or simple roots (sr)

        OUTPUT:
        - ``Output`` -- Dictionary; { Simple roots of Source : Root of Image } or { Fundamental weights of source : weight of Image }

        Caution: The mapping of the simple roots described by the input of ``Dictionary`` is a mapping of the form Torus(Image) --> Torus(Source)
           and therefore in the opposite direction as the underlying mapping Source --> Image.
        """
        Source = self
        sr_Source = Source.Basis('sr')
        fw_Source = Source.Basis('fw')

        assert isinstance( Image , Irreducible_Homogeneous_Variety ) , \
               'The input for ``Image`` need Image be an irreducible homogeneous variety.'
        WeightSpace_Image = Image.Parent_Group().Cartan_Type().root_system().weight_space()
        sr_Image = Image.Basis('sr')
        fw_Image = Image.Basis('fw')

        assert isinstance( Dictionary , dict ) , \
               'The input for ``Dictionary`` need Image be a dictionary.'

        srOutput = dict({})
        for Node in sr_Source.keys() :
            From = sr_Source[Node]
            assert From in Dictionary.keys() , \
                   'The '+str(Node)+'-th simple root of ```self``` is not a key in ```Dictionary```.'
            To = Dictionary[From]
            assert To in WeightSpace_Image , \
                   'The values of the dictionary need to be roots of weight space `'+str(WeightSpace_Image)+'`.'
            srOutput.update({ From : To })

        if Output_Type in self.BASIS_LABELLING['sr'] :
            return srOutput
        elif Output_Type in self.BASIS_LABELLING['fw'] :

            fwOutput = { fw_Source[Node] : sum([ Coefficient*srOutput[sr_Source[i]] for i , Coefficient in enumerate( Row , start=1 ) ])
                        for Node , Row in enumerate( Source.Parent_Group().Cartan_Type().cartan_matrix().inverse().transpose() , start=1 )
                       }
            return fwOutput
        else :
            raise ValueError('The input for ``Output_Type`` is inappropriate.')


    def N ( self ) -> int :
        """Returns the number ``N`` given in the realisation as Grassmannian."""
        G = self.Parent_Group()
        assert G.Is_Ordinary() , \
               ValueError('This method is only implemented for ordinary parent groups.')
        Cartan_Family = G.Cartan_Family()
        Cartan_Degree = G.Cartan_Degree()
        if   Cartan_Family == 'A' : return   Cartan_Degree + 1 # Grassmannians Gr(k,N) or Flag varieties FL( k1 , k2 , ... ; N )
        elif Cartan_Family == 'B' : return 2*Cartan_Degree + 1 # Orthogonal Grassmannians OGr(k,N) or orthogonal Flag varieties OFl( k1 , k2 , ... ; N )
        elif Cartan_Family == 'C' : return 2*Cartan_Degree     # Symplectic Grassmannians OGr(k,N) or symplectic Flag varieties SFl( k1 , k2 , ... ; N )
        elif Cartan_Family == 'D' : return 2*Cartan_Degree     # Orthogonal Grassmannians OGr(k,N) or orthogonal Flag varieties OFl( k1 , k2 , ... ; N )


    def Null_Weight ( self ) -> "Weight" :
        """Returns the null weight of ``self``."""
        fw = self.Basis('fw')
        return 0*fw[1]


    def Parent_Group ( self ) -> Irreducible_Cartan_Group :
        """Returns the parent group of ``self``."""
        return self.Parabolic_Subgroup().Parent_Group()


    def Parabolic_Subgroup ( self ) -> Parabolic_Subgroup_In_Irreducible_Cartan_Group :
        """Returns the attribute ``_Parabolic_Subgroup``."""
        return self._Parabolic_Subgroup


    def Second_Part ( self , Minimal_Degree :int =0 , Maximal_Degree :int =+infinity ) -> Iterator[ 'Irreducible_Equivariant_Vector_Bundle' ]  :
        assert self.Parabolic_Subgroup().Is_Maximal() , \
               ValueError('The method is only implemented for those homogeneous varieties where the parabolic subgroup is maximal.')
        assert isinstance( Minimal_Degree , Integer ) , \
               ValueError('The input for `Maximal_Degree` need to be an integer.')
        assert isinstance( Maximal_Degree , Integer ) or Maximal_Degree==+infinity , \
               ValueError('The input for `Maximal_Degree` need to be an integer or +infinity.')

        fw = self.Basis('fw')
        n = self.Parent_Group().Cartan_Degree()
        k = self.k()

        Degree = max([ Minimal_Degree , n-k+1 ])
        while Degree <= Maximal_Degree :
            Counter = 0
            for Counter , Coefficients in enumerate( IntegerListsLex( n=Degree , length=k-1 , max_part=n-k , max_slope=0 ) , start=1 ) :
                yield self.Irreducible_Equivariant_Vector_Bundle( sum([ self.Null_Weight() ] + [ Coefficient * fw[Node] for Node , Coefficient in enumerate( Coefficients , start=1 ) ])  )
            Degree += 1
            if not ( 0 < Counter ) : break


    # ..ToDo: Is not correct for Grassmannian!
    #def SemiSimplification_Of_Tangent_Bundle ( self ) :
    #    """Returns the semi-simplification of the tangent bundle on ``self``."""
    #    sr = self.Basis('sr')
    #    calTB = self.Zero_Vector_Bundle()
    #    for Counter1 , Root1 in enumerate( self.Available_Negative_Roots() ) :
    #        Comparison = [ self.Compare_Roots( Root1 , Root2 ) for Root2 in self.Available_Negative_Roots() ]
    #        if not ( '>' in Comparison[ : Counter1 ] or '>' in Comparison[ Counter1+1 : ] ) :
    #            # Write the root as linear combination of weights; as we worked with negative roots, we had the lowest weights and therefore need to mulitply with -1.
    #            Highest_Weight = -1*sum([ Coefficient * sr[Node] for Node , Coefficient in Root1 ])
    #            calTB += self.Equivariant_Vector_Bundle( Highest_Weight )
    #
    #    return calTB


    def Structure_Sheaf ( self , *Twists:tuple[int] ) -> "Irreducible_Equivariant_Vector_Bundle" :
        """Returns the structure sheaf or a twist of it."""
        if len(Twists) == 0 : Twists = tuple( len(self.Excluded_Nodes())*[0] )
        else :
            assert len(Twists) == len(self.Excluded_Nodes()) , \
                   ValueError('The input for ``Twists`` must be of the length ' + str( len(self.Excluded_Nodes()) ) +'(= number of excluded nodes).')
            for Twist_Counter , Twist in enumerate( Twists , start=1 ) :
                assert Twist in Integers() , 'The input data of the ' + str(Twist_Counter) + 'th twist is not an integer.'
        fw = self.Basis('fw')
        return self.Irreducible_Equivariant_Vector_Bundle( sum([ Twists[Counter]*fw[Node] for Counter , Node in enumerate( self.Excluded_Nodes() ) ]) )


    def Tautological_Subbundle ( self ) -> "Irreducible_Equivariant_Vector_Bundle" :
        """Returns the tautological subbundle, i.e. the dual of the equivariant vector bundle induced by the highest weight omega_1."""
        fw = self.Basis('fw')
        return self.Irreducible_Equivariant_Vector_Bundle( fw[1] ).Dual()


    def Trivial_Vector_Bundle ( self ) -> "Direct_Sum_Of_Equivariant_Vector_Bundles" :
        """Returns the trivial vector bundle, i.e. V^{ omega_1 } * calO."""
        fw = self.Basis('fw')
        V = self.Parent_Group().rmV( fw[1] )
        return self.calO().Multiply_By( V )


    # Synonym for the method ``Equivariant_Vector_Bundle``
    def VB ( self , *Input :tuple ) -> "Direct_Sum_Of_Equivariant_Vector_Bundle" :
        """Returns the equivariant vector bundle induced by the given highest weights."""
        return self.Equivariant_Vector_Bundle( *Input )


    def Zero_Vector_Bundle ( self ) -> "Direct_Sum_Of_Equivariant_Vector_Bundle" :
        """Returns the zero vector bundle over ``self``."""
        return self.Equivariant_Vector_Bundle()


In [6]:
class Grassmannian ( Irreducible_Homogeneous_Variety ) :

    def __init__( self , k :int , N :int ) -> None :
        """
        Initialize the Grassmannian Gr(k,N).

        INPUT:
        - ``k`` -- Integer ;
        - ``N`` -- Integer ;

        OUTPUT: None.
        """
        assert isinstance( N , Integer ) , \
               TypeError('The input for `N` must be an integer.')
        assert 2 <= N , \
               ValueError('The integer `N` must be equal to or greater than 2.')
        self._N = N

        assert isinstance( k , Integer ) , \
               TypeError('The input for `k` must be an integer.')
        assert k in [ 1 .. N-1 ] , \
               ValueError('The integer `k` need to be in the range '+str([ 1 .. N-1 ])+'.')
        self._k = k

        G = Irreducible_Cartan_Group( Cartan_Family='A' , Cartan_Degree=N-1 )
        P = G.Maximal_Parabolic_Subgroup( Excluded_Node=k )
        super( Grassmannian , self ).__init__( P )


    def __repr__ ( self ) -> tuple[ int ] :
        """Returns all attributes which are necessary to initialize the object."""
        return self.k() , self.N()


    def __str__ ( self , Output_Style='Long' ) -> str :
        """Returns a one-line string as short description."""
        if   Output_Style == 'Short' : return 'Gr('+str(self.k())+';'+str(self.N())+')'
        elif Output_Style == 'Long'  : return 'Grassmannian variety of '+str(self.k())+'-dimensional linear subspaces in a '+str(self.N())+'-dimensional ambient vector space.'
        else                         : raise ValueError('The input for ``Output_Style`` is inappropriate.')


    def Fano_Index ( self ) -> int :
        """Returns the Fano index of ``self``."""
        return self.N()


    def Fonarev_Collection ( self ) -> "Lefschetz_Collection" :
        """
        Returns Fonarev's (conjecturally full) minimal exceptional collection on ``self``.

        OUTPUT:
        - (Conjecturally full) minimal exceptional collection

        REFERENCE:
        - [Fon2012] Fonarev, A.: On minimal Lefschetz decompositions for Grassmannians
        """
        fw = self.Basis('fw')
        LC = self.Lefschetz_Collection( Starting_Block=[] , Support_Pattern='Trivial' )
        for YD , Orbit_Length in Young_Diagram.Minimal_Upper_Triangulars( self.N()-self.k() , self.k() ) :
            New_Object = self.Equivariant_Vector_Bundle( YD )
            LC += self.Lefschetz_Collection( Starting_Block=[New_Object] , Support_Pattern=tuple(Orbit_Length*[1]) )
        return LC


    def Kapranov_Collection ( self ) -> "Lefschetz_Collection" :
        """
        Returns Kapranov's full exceptional collection on ``self``.

        OUTPUT:
        - Full exceptional collection cU^alpha with n-k => alpha_1 => alpha_2 => ... => alpha_k => 0 (lexicographically orderd)

        REFERENCE:
        - [Kap1988] Kapranov, M. M.: On the derived categories of coherent sheaves on some homogeneous spaces. Invent. Math.92(1988), no.3, 479–508.
        """
        k = self.k()
        N = self.N()
        fw = self.Basis('fw')
        Weights = [ list(Weight)+[ 0 ] for Weight in IntegerListsLex( length=k , max_part=N-k , max_slope=0 ) ]
        Weights.reverse()
        Starting_Block = [ self.Irreducible_Equivariant_Vector_Bundle( sum([ self.Null_Weight() ] + [ (Weight[Node-1]-Weight[Node]) * fw[Node] for Node in [ 1 .. k ] ]) )
                           for Weight in Weights
                         ]
        Support_Pattern = 'Trivial'
        return self.Lefschetz_Collection( Starting_Block=Starting_Block , Support_Pattern=Support_Pattern )


class Projective_Space ( Grassmannian ) :

    def __init__( self , Dimension :int ) -> None :
        """
        Initialize the projective space of a given dimension.

        INPUT:
        - ``Dimension`` -- Integer ;

        OUTPUT: None.
        """
        assert isinstance( Dimension , Integer ) , \
               TypeError('The input for `Dimension` must be an integer.')
        assert 0 < Dimension , \
               ValueError('The dimension must be greater than zero.')

        super( Projective_Space , self ).__init__( k=1 , N=Dimension+1 )


    def __repr__ ( self ) -> int :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Dimension()


    def __str__ ( self , Output_Style='Long' ) -> str :
        """Returns a one-line string as short description."""
        if   Output_Style == 'Short' : return 'PP^'+str(self.Dimension())
        elif Output_Style == 'Long'  : return 'Projective space of dimension '+str(self.Dimension())+'.'
        else                         : raise ValueError('The input for ``Output_Style`` is inappropriate.')


    def Beilinson_Collection ( self ) -> "Lefschetz_Collection" :
        """
        Returns Beilinson's full exceptional collection on ``self``.

        OUTPUT:
        - Lefschetz collection with starting block ( O_X ) and support partition (self.Dimension()+1)*[ 1 ].

        REFERENCE:
        - [Bei1978] Beilinson, A. A.: Coherent sheaves on Pn and problems in linear algebra. Funktsional. Anal. i Prilozhen.12(1978), no.3, 68–69.
        """
        Starting_Block = [ self.calO() ]
        Support_Pattern = tuple( (self.Dimension()+1)*[ 1 ])
        return self.Lefschetz_Collection( Starting_Block=Starting_Block , Support_Pattern=Support_Pattern )


class Dual_Projective_Space ( Grassmannian ) :

    def __init__( self , Dimension :int ) -> None :
        """
        Initialize the dual projective space of a given dimension.

        INPUT:
        - ``Dimension`` -- Integer ;

        OUTPUT: None.
        """
        assert isinstance( Dimension , Integer ) , \
               TypeError('The input for `Dimension` must be an integer.')
        assert 0 < Dimension , \
               ValueError('The dimension must be greater than zero.')

        super( Dual_Projective_Space , self ).__init__( k=Dimension , N=Dimension+1 )


    def __repr__ ( self ) -> int :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Dimension()


    def __str__ ( self , Output_Style='Long' ) -> str :
        """Returns a one-line string as short description."""
        if   Output_Style == 'Short' : return 'PP^('+str(self.Dimension())+',v)'
        elif Output_Style == 'Long'  : return 'Dual projective space of dimension '+str(self.Dimension())+'.'
        else                         : raise ValueError('The input for ``Output_Style`` is inappropriate.')


In [7]:
class Orthogonal_Grassmannian ( Irreducible_Homogeneous_Variety ) :

    def __init__( self , k :int , N :int ) -> None :
        """
        Initialize the Grassmannian OGr(k,N).

        INPUT:
        - ``k`` -- Integer ;
        - ``N`` -- Integer ;

        OUTPUT: None.
        """
        assert isinstance( N , Integer ) , \
               TypeError('The input for `N` must be an integer.')
        assert 5 <= N , \
               ValueError('The integer `N` must be equal to or greater than 5.')

        assert isinstance( k , Integer ) , \
               TypeError('The input for `k` must be an integer.')
        assert k in [ 1 .. floor(N/2) ] , \
               ValueError('The integer `k` need to be in the range '+str([ 1 .. floor(N/2) ])+'.')

        if N % 2 == 0 :
            G = Irreducible_Cartan_Group( Cartan_Family='D' , Cartan_Degree=N/2 )
            if k in [ 1 .. N/2-2 ] : P = G.Maximal_Parabolic_Subgroup( Excluded_Node=k )
            elif k == N/2-1        : P = G.Parabolic_Subgroup( Excluded_Nodes={N/2-1,N/2} )
            elif k == N/2          : P = G.Maximal_Parabolic_Subgroup( Excluded_Node=N/2-1 )
        else          :
            G = Irreducible_Cartan_Group( Cartan_Family='B' , Cartan_Degree=floor(N/2) )
            P = G.Maximal_Parabolic_Subgroup( Excluded_Node=k )

        super( Orthogonal_Grassmannian , self ).__init__( P )


    def __repr__ ( self ) -> tuple[ int ] :
        """Returns all attributes which are necessary to initialize the object."""
        return self.k() , self.N()


    def __str__ ( self , Output_Style='Long' ) -> str :
        """Returns a one-line string as short description."""
        if   Output_Style == 'Short' : return 'OGr('+str(self.k())+';'+str(self.N())+')'
        elif Output_Style == 'Long'  : return 'Orthogonal grassmannian variety of '+str(self.k())+'-dimensional isotropic linear subspaces' + \
                                              ' in a '+str(self.N())+'-dimensional ambient vector space.'
        else                         : raise ValueError('The input for ``Output_Style`` is inappropriate.')


    # Synonym for the method ``Spinor_Bundle``
    def calS ( self ) -> "Irreducible_Equivariant_Vector_Bundle" :
        """Returns the spinor bundle ."""
        return self.Spinor_Bundle()


    def Fano_Index ( self ) -> int :
        """Returns the Fano index of ``self``."""
        for Node , Fano_Index in super( Orthogonal_Grassmannian , self ).Fano_Index() :
            if Node == self.k() : return Fano_Index
        raise ValueError('PROBLEM! We could not compute the Fano index.')


    def Tautological_Collection ( self ) -> "Lefschetz_Collection" :
        n = self.Parent_Group().Cartan_Degree()
        k = self.k()
        fw = self.Basis('fw')

        Starting_Block = []
        for d in [ 0 .. n-k ] :
            Starting_Block += [ self.calU( sum([ self.Null_Weight() ] + [ Coefficient*fw[Node] for Node , Coefficient in enumerate( Coefficients , start=1 ) ]) )
                                for Coefficients in IntegerListsLex( n=d , length=k-1 )
                              ]
        Support_Partition = self.Fano_Index() * [ len(Starting_Block) ]
        return self.Lefschetz_Collection( Starting_Block=Starting_Block , Support_Pattern=Support_Partition )


    def Spinor_Bundle ( self , Parity : int or str or None =None ) -> "Irreducible_Equivariant_Vector_Bundle" :
        """Returns the spinor bundle."""
        Cartan_Family = self.Cartan_Family()
        n = self.Cartan_Degree()
        fw = self.Basis('fw')
        if   Cartan_Family == 'B' :
            return self.calU( fw[n] )

        elif Cartan_Family == 'D' :
            if   Parity in [  1 , '+' ] : return self.calU( fw[n-1] )
            elif Parity in [ -1 , '-' ] : return self.calU( fw[n] )
            else               : raise ValueError('For the Cartan Family D, there need to be a parity `+` or `-`.')

        else :
            raise ValueError('Conflict! There are only spinor bundles for Cartan families B_n and D_n respectively.')


    def Spinor_Collection ( self ) -> "Lefschetz_Collection" :
        Cartan_Family = self.Parent_Group().Cartan_Family()
        n = self.Parent_Group().Cartan_Degree()
        k = self.k()
        fw = self.Basis('fw')
        LC = self.Lefschetz_Collection( Starting_Block=[] , Support_Pattern='Trivial' )
        if   Cartan_Family == 'B' :
            Degree = 0
            Keep_Running = True
            while Keep_Running :
                Current_Component = self.calU( Degree*fw[1]+fw[n] )
                if Degree == 0 : Orbit = Current_Component.Maximal_Numerically_Exceptional_Orbit()
                else           : Orbit = Current_Component.Extend_Equivariantly_By( Previous_Component ).Maximal_Numerically_Exceptional_Orbit()
                Candidate = LC + Orbit
                if 0 < len(Orbit) and Candidate.Is_Numerically_Exceptional() :
                    LC = Candidate
                    Degree += 1
                    Previous_Component = Current_Component
                else              :
                    Keep_Running = False

        elif Cartan_Family == 'D' :
            pass

        return LC


    def Spinor_Filtration ( self , Parity :str or None =None , Factor :"Equivariant_Vector_Bundle" or None =None ) -> Iterator[ str ] :
        """
        Returns the filtration on the spinor bundle.

        REFERENCE:
        - [Kuz08a] Kuznetsov, Alexander: Exceptional collections for Grassmannians of isotropic lines.
        """
        if Factor == None : Factor = self.calO()
        assert isinstance( Factor , Equivariant_Vector_Bundle ) , \
               'The input for `Factor` need to be an equivariant vector bundle.'
        Cartan_Family = self.Cartan_Family()
        n = self.Cartan_Degree()
        k = self.k()
        fw = self.Basis('fw')
        if   Cartan_Family == 'B' :
            calS = self.calS()
            calU = self.calU()
            yield 'F_'+str(k+1)+' = 0'
            for i in range( k , -1 , -1 ) :
                Quotient = str( calS * calU.Exterior_Power(i) * Factor )
                yield '0 --> F_'+str(i+1)+' --> F_'+str(i)+' --> '+str(Quotient)+' --> 0'
            yield 'F_0 = '+str( Factor.Multiply_By( calS.H0() ) )

        elif Cartan_Family == 'D' :
            if   Parity in [  1 , '+' ] : ParityStr = '+' ; ParityInt = 1
            elif Parity in [ -1 , '-' ] : ParityStr = '-' ; ParityInt = -1
            else                        : raise ValueError('For the Cartan Family D, there need to be a parity in '+str([ 1 , '+' ])+' or in '+str([ -1 , '-' ])+'.')
            yield 'F^'+ParityStr+'_'+str(k+1)+' = 0'
            for i in range( k , -1 , -1 ) :
                Quotient = str( self.calS( ParityInt * (-1)^i ) * calU.Exterior_Power(i) * Factor )
                yield '0 --> F^'+ParityStr+'_'+str(i+1)+' --> F^'+ParityStr+'_'+str(i)+' --> '+str(Quotient)+' --> 0'
            yield 'F_0 = '+str( Factor.Multiply_By( self.calS(Parity).H0() ) )

        else :
            raise ValueError('Conflict! There are only spinor bundles for Cartan families B_n and D_n respectively.')


class Quadratic_Space ( Orthogonal_Grassmannian ) :

    def __init__( self , Dimension :int ) -> None :
        """
        Initialize the quadratic space of a given dimension.

        INPUT:
        - ``Dimension`` -- Integer ;

        OUTPUT: None.
        """
        assert isinstance( Dimension , Integer ) , \
               TypeError('The input for `Dimension` must be an integer.')
        assert 0 < Dimension , \
               ValueError('The dimension must be greater than zero.')

        super( Quadratic_Space , self ).__init__( k=1 , N=Dimension+2 )


    def __repr__ ( self ) -> int :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Dimension()


    def __str__ ( self , Output_Style='Long' ) -> str :
        """Returns a one-line string as short description."""
        if   Output_Style == 'Short' : return 'Q^'+str(self.Dimension())
        elif Output_Style == 'Long'  : return 'Quadratic space of dimension '+str(self.Dimension())+'.'
        else                         : raise ValueError('The input for ``Output_Style`` is inappropriate.')

In [8]:
class Direct_Sum_Of_Homogeneous_Varieties ( Direct_Sum_Of_Algebraic_Structures , Homogeneous_Variety ) :
    pass